## 先决条件

- 在 Azure 中运行的 AKS 群集。 如果以前没有使用过群集扩展，则需要[注册 KubernetesConfiguration 服务提供程序](https://learn.microsoft.com/zh-cn/azure/aks/dapr#register-the-kubernetesconfiguration-resource-provider)。
- 或者 Arc Kubernetes 群集已启动并正在运行。 按照[将现有 Kubernetes 群集连接到 Azure Arc](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/quickstart-connect-cluster) 中的说明操作。
    - 如果群集是 Azure RedHat OpenShift (ARO) 服务群集或 OpenShift 容器平台 (OCP) 群集，则必须满足[配置 Kubernetes 群集的参考](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-kubernetes?view=azureml-api-2#prerequisites-for-aro-or-ocp-clusters)一文中所述的其他先决条件步骤。
- 出于生产目的，Kubernetes 群集必须至少具有 4 个 vCPU 核心和 14 GB 内存。 有关资源及群集大小建议的详细信息，请参阅[建议的资源计划](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-kubernetes?view=azureml-api-2)。
- 在出站代理服务器或防火墙后面运行的群集需要额外的**网络配置**。
- 安装或升级 Azure CLI，确保其版本为 2.24.0 或更高版本。
- 安装或升级 Azure CLI 扩展 `k8s-extension`，确保其版本为 1.2.3 或更高版本。

## 限制

- Azure 机器学习不支持[将服务主体与 AKS 配合使用](https://learn.microsoft.com/zh-cn/azure/aks/kubernetes-service-principal)。 AKS 群集必须改用托管标识。 同时支持系统分配的托管标识和用户分配的托管标识。 有关详细信息，请参阅[在 Azure Kubernetes 服务中使用托管标识](https://learn.microsoft.com/zh-cn/azure/aks/use-managed-identity)。
    - 将 AKS 群集使用的服务主体转换为使用托管标识后，在安装扩展之前，需要删除并重新创建所有节点池，而不是直接更新。
- Azure 机器学习不支持为 AKS [禁用本地帐户](https://learn.microsoft.com/zh-cn/azure/aks/manage-local-accounts-managed-azure-ad#disable-local-accounts)。 部署 AKS 群集时，本地帐户默认处于启用状态。
- 如果 AKS 群集已启用[授权 IP 范围以访问 API 服务器](https://learn.microsoft.com/zh-cn/azure/aks/api-server-authorized-ip-ranges)，请为该 AKS 群集启用 Azure 机器学习控制平面 IP 范围。 Azure 机器学习控制平面跨配对区域部署。 如果没有 API 服务器的访问权限，则无法部署机器学习 Pod。 在 AKS 群集中启用 IP 范围时，请对两个[配对区域](https://www.microsoft.com/en-us/download/details.aspx?id=56519)都使用 [IP 范围](https://learn.microsoft.com/zh-cn/azure/reliability/cross-region-replication-azure)。
- Azure 机器学习不支持跨订阅附加 AKS 群集。 如果 AKS 群集位于其他订阅中，必须先[将其连接到 Azure-Arc](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/quickstart-connect-cluster)，并在与 Azure 机器学习工作区相同的订阅中指定。
- Azure 机器学习不保证支持 AKS 中的所有预览阶段功能。 例如，不支持 [Microsoft Entra Pod 标识](https://learn.microsoft.com/zh-cn/azure/aks/use-azure-ad-pod-identity)。
- 如果已按照 [Azure 机器学习 AKS v1 文档](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-create-attach-kubernetes?view=azureml-api-1&preserve-view=true)中的步骤创建或附加 AKS 作为推理群集，请在继续下一步之前使用以下链接[清理旧版 azureml-fe 相关资源](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-create-attach-kubernetes?view=azureml-api-1&preserve-view=true#delete-azureml-fe-related-resources)。


## 查看 Azure 机器学习扩展配置设置

可以使用 Azure 机器学习 CLI 命令 `k8s-extension create` 来部署 Azure 机器学习扩展。 借助 CLI `k8s-extension create`，你可以使用 `key=value` 或 `--config` 参数以 `--config-protected` 格式指定一组配置设置。 以下列出了 Azure 机器学习扩展部署期间要指定的可用配置设置。

|配置设置键名|说明|培训|推理|训练和推理|
|---|---|---|---|---|
|`enableTraining`|`True` 或 `False`，默认为 `False`。 对于支持机器学习模型训练和批量评分的 Azure 机器学习扩展部署，此项必须设置为 。|**✓**|空值|**✓**|
|`enableInference`|`True` 或 `False`，默认为 `False`。 对于支持机器学习推理的 Azure 机器学习扩展部署，此项必须设置为 。|空值|**✓**|**✓**|
|`allowInsecureConnections`|`True` 或 `False`，默认为 `False`。 可以设置为 以将推理 HTTP 终结点用于开发或测试。|空值|可选|可选|
|`inferenceRouterServiceType`|`loadBalancer`、`nodePort` 或 `clusterIP`。 必需（如果 ）。|空值|**✓**|**✓**|
|`internalLoadBalancerProvider`|此配置目前仅适用于 Azure Kubernetes 服务 (AKS) 群集。 设置为 `azure` 以允许推理路由器使用内部负载均衡器。|空值|可选|可选|
|`sslSecret`|Kubernetes 机密在 `azureml` 命名空间中的名称。 此配置用于存储 `cert.pem`（PEM 编码的 TLS/SSL 证书）和 `key.pem`（PEM 编码的 TLS/SSL 密钥），当 `allowInsecureConnections` 设置为 `False` 时，这是支持推理 HTTPS 终结点所必需的。 有关 `sslSecret` 的示例 YAML 定义，请参阅[配置 sslSecret](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-secure-kubernetes-online-endpoint?view=azureml-api-2)。 使用此配置或 `sslCertPemFile` 和 `sslKeyPemFile` 保护的配置设置的组合。|空值|可选|可选|
|`sslCname`|推理 HTTPS 终结点使用 TLS/SSL CNAME。 必需（如果 ）|空值|可选|可选|
|`inferenceRouterHA`|`True` 或 `False`，默认为 `True`。 默认情况下，Azure 机器学习扩展会部署 3 个推理路由器副本来实现高可用性，这需要在群集中至少提供 3 个工作器节点。 如果群集的工作器节点少于 3 个，则将此项设置为 `False`。在这种情况下只会部署一个推理路由器服务。|空值|可选|可选|
|`nodeSelector`|默认情况下，部署的 Kubernetes 资源和机器学习工作负载将随机部署到群集的一个或多个节点，而 DaemonSet 资源将部署到所有节点。 若要将扩展部署和训练/推理工作负载限制为具有标签 `key1=value1` 和 `key2=value2` 的特定节点，请相应地使用 `nodeSelector.key1=value1`、`nodeSelector.key2=value2`。|可选|可选|可选|
|`installNvidiaDevicePlugin`|`True` 或 `False`，默认为 `False`。 NVIDIA GPU 硬件上的 ML 工作负载需要 [NVIDIA 设备插件](https://github.com/NVIDIA/k8s-device-plugin#nvidia-device-plugin-for-kubernetes)。 默认情况下，无论 Kubernetes 群集是否包含 GPU 硬件，Azure 机器学习扩展部署都会安装 NVIDIA 设备插件。 用户可以将此设置指定为 `True` 以进行安装，但请确保满足[先决条件](https://github.com/NVIDIA/k8s-device-plugin#prerequisites)。|可选|可选|可选|
|`installPromOp`|`True` 或 `False`，默认为 `True`。 Azure 机器学习扩展需要 prometheus 运算符来管理 prometheus。 设置为 `False` 以重用现有的 prometheus operator。 有关重用现有 prometheus operator 的详细信息，请参阅[重用 prometheus operator](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-troubleshoot-kubernetes-extension?view=azureml-api-2#prometheus-operator)|可选|可选|可选|
|`installVolcano`|`True` 或 `False`，默认为 `True`。 Azure 机器学习扩展需要使用 volcano 计划程序来计划作业。 请设置为 `False` 以重用现有的 volcano 计划程序。 有关重用现有 volcano scheduler 的详细信息，请参阅[重用 volcano scheduler](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-troubleshoot-kubernetes-extension?view=azureml-api-2#volcano-scheduler)|可选|空值|可选|
|`installDcgmExporter`|`True` 或 `False`，默认为 `False`。 Dcgm-exporter 可以公开 Azure 机器学习工作负载的 GPU 指标，可以在 Azure 门户中对其进行监视。 将 `installDcgmExporter` 设置为 `True` 以安装 dcgm-exporter。 但若要使用自己的 dcgm-exporter，请参阅 [DCGM 导出程序](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-troubleshoot-kubernetes-extension?view=azureml-api-2#dcgm-exporter)|可选|可选|可选|

|受保护配置设置键名称|说明|培训|推理|训练和推理|
|---|---|---|---|---|
|`sslCertPemFile`、`sslKeyPemFile`|当 `allowInsecureConnections` 设置为 False 时，TLS/SSL 证书和密钥文件（经过 PEM 编码）的路径是支持推理 HTTPS 终结点的 Azure 机器学习扩展部署所必需的。 注意：不支持带有密码保护的 PEM 文件|空值|可选|可选|

如配置设置表所示，不同配置设置的组合支持针对不同的 ML 工作负载场景部署 Azure 机器学习扩展：

- 对于训练作业和批处理推理工作负载，请指定 `enableTraining=True`
- 对于仅推理工作负载，请指定 `enableInference=True`
- 对于所有类型的 ML 工作负载，请同时指定 `enableTraining=True` 和 `enableInference=True`

如果计划为实时推理工作负载部署 Azure 机器学习扩展并想要指定 `enableInference=True`，请注意以下与实时推理工作负载相关的配置设置：

- 实时推理支持需要 `azureml-fe` 路由器服务，并需要为 `inferenceRouterServiceType` 指定 `azureml-fe` 配置设置。 `azureml-fe` 可以采用以下 `inferenceRouterServiceType` 之一部署：
    - 键入 `LoadBalancer`。 使用云提供商的负载均衡器在外部公开 `azureml-fe`。 若要指定此值，请确保群集支持负载均衡器预配。 请注意，大多数本地 Kubernetes 群集可能不支持外部负载均衡器。
    - 键入 `NodePort`。 在静态端口上的每个节点 IP 中公开 `azureml-fe`。 可以通过请求 `azureml-fe` 从群集外部联系 `<NodeIP>:<NodePort>`。 使用 `NodePort` 还可为 `azureml-fe` 设置你自己的负载均衡解决方案和 TLS/SSL 终止。 有关如何设置自己的入口的更多详细信息，请参阅 [通过 HTTP 或 HTTPS 将其他入口控制器与 Azure 机器学习扩展集成](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-kubernetes?view=azureml-api-2#integrate-other-ingress-controller-with-azure-machine-learning-extension-over-http-or-https)。
    - 键入 `ClusterIP`。 在群集内部 IP 中公开 `azureml-fe`，这样就只能从群集内部访问 `azureml-fe`。 若要让 `azureml-fe` 为来自群集外部的推理请求提供服务，你需要为 `azureml-fe` 设置自己的负载均衡解决方案和 TLS/SSL终止。 有关如何设置自己的入口的更多详细信息，请参阅 [通过 HTTP 或 HTTPS 将其他入口控制器与 Azure 机器学习扩展集成](https://learn.microsoft.com/zh-cn/azure/machine-learning/reference-kubernetes?view=azureml-api-2#integrate-other-ingress-controller-with-azure-machine-learning-extension-over-http-or-https)。
- 为确保 `azureml-fe` 路由服务的高可用性，Azure 机器学习扩展部署默认将为包含 3 个或更多节点的群集创建 3 个 `azureml-fe` 副本。 如果群集的节点数少于 3 个，请设置 。
- 还需要考虑使用 HTTPS 来限制对模型终结点的访问，以及保护客户端提交的数据。 为此，需要指定 `sslSecret` 配置设置或者 `sslKeyPemFile` 和 `sslCertPemFile` 配置保护设置的组合。
- 默认情况下，Azure 机器学习扩展部署要求为 HTTPS 支持提供配置设置。 对于开发或测试，可以通过配置设置 方便地提供 HTTP 支持。


## Azure 机器学习扩展部署 - CLI 示例

若要使用 CLI 部署 Azure 机器学习扩展，请使用 `az k8s-extension create` 命令传入强制参数的值。

我们列出了 4 种典型的扩展部署场景以供参考。 若要为生产用途部署扩展，请仔细阅读[配置设置](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-kubernetes-extension?view=azureml-api-2&tabs=deploy-extension-with-cli#review-azure-machine-learning-extension-configuration-settings)的完整列表。

- 将 Azure 中的 AKS 群集用于快速进行概念证明，以运行各种 ML 工作负载，即运行训练作业或将模型部署为联机/批处理终结点
    
    对于 AKS 群集上的 Azure 机器学习扩展部署，请确保为 `managedClusters` 参数指定 `--cluster-type` 值。 运行以下 Azure CLI 命令以部署 Azure 机器学习扩展：

In [ ]:
extension_name="text-aks-ex"
cluster_name="text-aks-cluster"
resource_group="text-aks-rg"

In [ ]:
!az k8s-extension create --name {extension_name} --extension-type Microsoft.AzureML.Kubernetes --config enableTraining=True enableInference=True inferenceRouterServiceType=LoadBalancer allowInsecureConnections=True InferenceRouterHA=False --cluster-type managedClusters --cluster-name {cluster_name} --resource-group {resource_group} --scope cluster

- **将 Azure 外部的 Arc Kubernetes 群集用于快速进行概念证明，以仅运行训练作业**
    
    对于 [Arc Kubernetes](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/overview) 群集上的 Azure 机器学习扩展部署，需要为 `connectedClusters` 参数指定 `--cluster-type` 值。 运行以下 Azure CLI 命令以部署 Azure 机器学习扩展：
    

In [ ]:
!az k8s-extension create --name {extension_name} --extension-type Microsoft.AzureML.Kubernetes --config enableTraining=True --cluster-type connectedClusters --cluster-name {extension_name} --resource-group {resource_group} --scope cluster

- **在 Azure 中为生产训练和推理工作负载启用 AKS 群集** 对于 AKS 上的 Azure 机器学习扩展部署，请确保为 `managedClusters` 参数指定 `--cluster-type` 值。 假设群集包含 3 个以上的节点，并且你需要使用 Azure 公共负载均衡器和 HTTPS 来提供推理工作负载支持。 运行以下 Azure CLI 命令以部署 Azure 机器学习扩展：
  - ssl_cname: 推理 HTTPS 终结点使用 TLS/SSL CNAME
  - ssl_cert_pem_file: TLS/SSL 证书文件（经过 PEM 编码）的路径
  - ssl_key_pem_file: TLS/SSL 密钥文件（经过 PEM 编码）的路径

In [ ]:
ssl_cname="text-aks-ssl-cname"
ssl_cert_pem_file="text-aks-ssl-cert-PEM"
ssl_key_pem_file="text-aks-ssl-cert-KEY"

In [ ]:
!az k8s-extension create --name {extension_name} --extension-type Microsoft.AzureML.Kubernetes --config enableTraining=True enableInference=True inferenceRouterServiceType=LoadBalancer sslCname={ssl_cname} --config-protected sslCertPemFile={ssl_cert_pem_file} sslKeyPemFile={ssl_key_pem_file} --cluster-type managedClusters --cluster-name {cluster_name} --resource-group {resource_group} --scope cluster

- 使用 NVIDIA GPU 随时随地为生产训练和推理工作负载启用 [Arc Kubernetes](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/overview) 群集
    
    对于 [Arc Kubernetes](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/overview) 群集上的 Azure 机器学习扩展部署，请确保为 `connectedClusters` 参数指定 `--cluster-type` 值。 假设群集包含 3 个以上的节点，并且你需要使用 NodePort 服务类型和 HTTPS 来提供推理工作负载支持，请运行以下 Azure CLI 命令来部署 Azure 机器学习扩展：

    - `installNvidiaDevicePlugin` 安装 NVIDIA 设备插件
    - `installDcgmExporter` 安装 dcgm-exporter


In [ ]:
!az k8s-extension create --name {extension_name} --extension-type Microsoft.AzureML.Kubernetes --config enableTraining=True enableInference=True inferenceRouterServiceType=NodePort sslCname={ssl_cname} installNvidiaDevicePlugin=True installDcgmExporter=True --config-protected sslCertPemFile={ssl_cert_pem_file} sslKeyPemFile={ssl_key_pem_file} --cluster-type connectedClusters --cluster-name {cluster_name} --resource-group {resource_group} --scope cluster

### 验证 Azure 机器学习扩展部署

1. 运行以下 CLI 命令来检查 Azure 机器学习扩展详细信息：

In [ ]:
!az k8s-extension show --name {extension_name} --cluster-type connectedClusters --cluster-name {cluster_name} --resource-group {resource_group}


2. 在响应中，查找 "name" 和 "provisioningState": "Succeeded"。 请注意，前几分钟的响应可能显示 "provisioningState": "Pending"。
    
3. 如果 provisioningState 显示 Succeeded，请在将 kubeconfig 文件指向你的群集的情况下，在计算机上运行以下命令，以检查 azureml 命名空间下的所有 pod 是否都处于“Running”状态：

In [ ]:
!kubectl get pods -n azureml

## 查看 Azure 机器学习扩展组件

Azure 机器学习扩展部署完成后，可以使用 `kubectl get deployments -n azureml` 查看群集中创建的资源列表。 其中通常包含每个指定配置设置的以下资源的子集。


|资源名称|资源类型|培训|推理|训练和推理|说明|与云通信|
|---|---|---|---|---|---|---|
|中继服务器|Kubernetes 部署|**✓**|**✓**|**✓**|中继服务器仅为 Arc Kubernetes 群集创建，且不会在 AKS 群集中创建。 中继服务器与 Azure 中继配合使用，以便与云服务通信。|从云服务接收作业创建和模型部署请求；将作业状态与云服务同步。|
|网关|Kubernetes 部署|**✓**|**✓**|**✓**|网关用于来回通信和发送数据。|将节点和群集资源信息发送到云服务。|
|aml-operator|Kubernetes 部署|**✓**|空值|**✓**|管理训练作业的生命周期。|与云令牌服务交换令牌，以便对 Azure 容器注册表进行身份验证和授权。|
|metrics-controller-manager|Kubernetes 部署|**✓**|**✓**|**✓**|管理 Prometheus 的配置|空值|
|{EXTENSION-NAME}-kube-state-metrics|Kubernetes 部署|**✓**|**✓**|**✓**|将群集相关的指标导出到 Prometheus。|空值|
|{EXTENSION-NAME}-prometheus-operator|Kubernetes 部署|可选|可选|可选|提供 Prometheus 和相关监视组件的 Kubernetes 本机部署和管理。|空值|
|amlarc-identity-controller|Kubernetes 部署|空值|**✓**|**✓**|通过托管标识请求和续订 Azure Blob/Azure 容器注册表令牌。|与云令牌服务交换令牌，以便对推理/模型部署使用的 Azure 容器注册表和 Azure Blob 进行身份验证和授权。|
|amlarc-identity-proxy|Kubernetes 部署|空值|**✓**|**✓**|通过托管标识请求和续订 Azure Blob/Azure 容器注册表令牌。|与云令牌服务交换令牌，以便对推理/模型部署使用的 Azure 容器注册表和 Azure Blob 进行身份验证和授权。|
|azureml-fe-v2|Kubernetes 部署|空值|**✓**|**✓**|用于将传入推理请求路由到已部署服务的前端组件。|将服务日志发送到 Azure 存储。|
|inference-operator-controller-manager|Kubernetes 部署|空值|**✓**|**✓**|管理推理终结点的生命周期。|空值|
|火山入院|Kubernetes 部署|可选|空值|可选|Volcano 许可 Webhook。|空值|
|火山控制器|Kubernetes 部署|可选|空值|可选|管理 Azure 机器学习训练作业 pod 的生命周期。|空值|
|volcano-scheduler|Kubernetes 部署|可选|空值|可选|用于执行群集内作业计划。|空值|
|fluent-bit|Kubernetes DaemonSet|**✓**|**✓**|**✓**|收集组件的系统日志。|将组件的系统日志上传到云。|
|{EXTENSION-NAME}-dcgm-exporter|Kubernetes daemonset|可选|可选|可选|dcgm-exporter 公开 Prometheus 的 GPU 指标。|空值|
|nvidia-device-plugin-daemonset|Kubernetes daemonset|可选|可选|可选|nvidia-device-plugin-daemonset 在群集的每个节点上公开 GPU|空值|
|prometheus-prom-prometheus|Kubernetes statefulset|**✓**|**✓**|**✓**|收集作业指标并将其发送到云。|将 CPU/GPU/内存利用率等作业指标发送到云。|

 重要

- Azure 中继资源是在 Arc 群集资源所在的同一资源组下创建的。 可以使用这些资源与 Kubernetes 群集通信，但修改它们会破坏附加的计算目标。
- 默认情况下，Kubernetes 部署资源将随机部署到群集上的 1 个或多个节点，而守护程序集资源将部署到所有节点。 若要将扩展部署限制为特定的节点，请按`nodeSelector`所述使用 [](https://learn.microsoft.com/zh-cn/azure/machine-learning/how-to-deploy-kubernetes-extension?view=azureml-api-2&tabs=deploy-extension-with-cli#review-azure-machine-learning-extension-configuration-settings)配置设置。

 备注

- {EXTENSION-NAME}: 是由 CLI 命令指定的扩展名。

### 管理 Azure 机器学习扩展

更新、列出、显示和删除 Azure 机器学习扩展。

- 对于未连接 Azure Arc 的 AKS 群集，请参阅[部署和管理群集扩展](https://learn.microsoft.com/zh-cn/azure/aks/deploy-extensions-az-cli)。
- 有关已启用 Azure Arc 的 Kubernetes，请参阅[部署和管理已启用 Azure Arc 的 Kubernetes 群集扩展](https://learn.microsoft.com/zh-cn/azure/azure-arc/kubernetes/extensions)。